# CODE FILE FOR ROBERTA + LSTM + GCN + CONCEPTNET  

# Improved Deception Detection Model (With Conceptnet)

In this code file , we have implemented an end‐to‐end revised version of the deception detection code. In this version, we incorporate several improvements inspired by the research (e.g. Peskov et al. 2020) and related work on deception detection. In particular, we:

- Introduce a new context encoder module that uses a bidirectional LSTM to summarize token‐level representations of the conversation context.
- Fuse five streams of information: the current message representation (from RoBERTa), the contextual summary from the LSTM, graph features derived from conversation (via two graph attention layers), and game score features (as a proxy for power dynamics) and ConceptNet embeddings . 
- Use an ensemble of classifier heads (with learnable weights) to stabilize predictions.

Due to limited data size, early transformer layers are frozen and extra regularization is applied.

In [7]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from collections import Counter, defaultdict
from sklearn.metrics import f1_score, confusion_matrix, classification_report
import pandas as pd
from transformers import AutoModel, AutoTokenizer, get_linear_schedule_with_warmup, RobertaModel
import spacy

# Set seeds for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

Using device: cuda


In [8]:

# Data paths
TRAIN_PATH = r"/kaggle/input/nlp-proj-data/Data/train.jsonl"
VAL_PATH = r"/kaggle/input/nlp-proj-data/Data/validation.jsonl"
TEST_PATH = r"/kaggle/input/nlp-proj-data/Data/test.jsonl"

# Model and training parameters
TRANSFORMER_MODEL = "roberta-base"  # Using RoBERTa base
BATCH_SIZE = 32
EPOCHS = 5
LR = 5e-6
USE_GAME_SCORES = True
OVERSAMPLING_FACTOR = 30  # Oversampling factor (tune if necessary)
TRUTH_FOCAL_WEIGHT = 4.0  # Additional weight for truth class during loss calculation
EARLY_STOPPING_PATIENCE = 5
GRADIENT_ACCUMULATION_STEPS = 2


In [9]:
# Load spaCy model for entity extraction
nlp = spacy.load("en_core_web_sm")

def load_numberbatch(path="/kaggle/input/concept-net/numberbatch-en.txt"):
    """Load ConceptNet Numberbatch embeddings into a dictionary."""
    embeddings = {}
    with open(path, 'r', encoding='utf-8') as f:
        next(f)  # Skip header line if present
        for line in f:
            parts = line.strip().split()
            key = parts[0]  # e.g., '/c/en/apple'
            vector = np.array(parts[1:], dtype=np.float32)
            embeddings[key] = vector
    return embeddings

class EnhancedDeceptionDataset(Dataset):
    def __init__(self, path, tokenizer, max_len=128, use_game_scores=True):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.use_game_scores = use_game_scores
        self.texts = []
        self.labels = []
        self.scores = []
        self.conversation_ids = []
        self.message_positions = []
        self.prior_context = []
        self.conceptnet_features = []  # Store ConceptNet features
        
        # Load Numberbatch embeddings
        self.numberbatch_embeddings = load_numberbatch()
        self.conceptnet_dim = 300  # Numberbatch embedding size
        
        with open(path, 'r', encoding='utf-8') as f:
            conv_id = 0
            for line in f:
                data = json.loads(line.strip())
                messages = data.get('messages', [])
                labels = data.get('sender_labels', [])
                game_scores = data.get('game_score_delta', None) if use_game_scores else None
                if game_scores is None:
                    game_scores = [0] * len(messages)
                
                for pos, (msg, label, score) in enumerate(zip(messages, labels, game_scores)):
                    if label in [True, False, "true", "false", "True", "False"]:
                        if isinstance(label, str):
                            is_lie = label.lower() == "true"
                        else:
                            is_lie = label
                        self.texts.append(msg)
                        self.labels.append(1 if is_lie else 0)
                        self.scores.append(float(score))
                        self.conversation_ids.append(conv_id)
                        self.message_positions.append(pos)
                        context = ""
                        if pos > 0:
                            context_msgs = messages[max(0, pos-2):pos]
                            context = " [SEP] ".join(context_msgs)
                        self.prior_context.append(context)
                        
                        # Extract entities and compute ConceptNet features
                        entities = self.extract_entities(msg)
                        embeddings = [self.get_numberbatch_embedding(ent) for ent in entities]
                        if embeddings:
                            avg_embedding = np.mean(embeddings, axis=0)
                        else:
                            avg_embedding = np.zeros(self.conceptnet_dim)
                        self.conceptnet_features.append(avg_embedding)
                conv_id += 1
        
        self.class_counts = Counter(self.labels)
        total = len(self.labels)
        self.truth_indices = [i for i, label in enumerate(self.labels) if label == 0]
        self.lie_indices = [i for i, label in enumerate(self.labels) if label == 1]
        self.conv_to_msgs = defaultdict(list)
        for i, cid in enumerate(self.conversation_ids):
            self.conv_to_msgs[cid].append(i)
        
        print(f"Dataset loaded from {path}")
        print(f"Total messages: {total}")
        for label, count in sorted(self.class_counts.items()):
            label_name = "Truth" if label == 0 else "Lie"
            print(f"{label_name}: {count} ({count/total*100:.2f}%)")

    def extract_entities(self, text):
        """Extract entities or nouns from text using spaCy."""
        doc = nlp(text)
        entities = [ent.text.replace(' ', '_').lower() for ent in doc.ents]
        if not entities:
            entities = [token.text.replace(' ', '_').lower() for token in doc if token.pos_ in ["NOUN", "PROPN"]]
        return entities[:5]  # Limit to top 5 entities

    def get_numberbatch_embedding(self, entity):
        """Retrieve Numberbatch embedding for an entity."""
        key = f"/c/en/{entity}"
        return self.numberbatch_embeddings.get(key, np.zeros(self.conceptnet_dim))

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        context = self.prior_context[idx]
        if context:
            context_encoding = self.tokenizer(
                context,
                max_length=self.max_len,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
        else:
            context_encoding = {
                'input_ids': torch.zeros((1, self.max_len), dtype=torch.long),
                'attention_mask': torch.zeros((1, self.max_len), dtype=torch.long)
            }
        
        return {
            'text': self.texts[idx],
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'context_input_ids': context_encoding['input_ids'].flatten(),
            'context_attention_mask': context_encoding['attention_mask'].flatten(),
            'label': torch.tensor(self.labels[idx], dtype=torch.long),
            'score': torch.tensor(self.scores[idx], dtype=torch.float),
            'conv_id': self.conversation_ids[idx],
            'position': self.message_positions[idx],
            'relative_positions': [],
            'conceptnet_features': torch.tensor(self.conceptnet_features[idx], dtype=torch.float)
        }

    def __len__(self):
        return len(self.labels)

class EnhancedBalancedSampler(torch.utils.data.Sampler):
    def __init__(self, dataset, oversample_factor=15):
        self.dataset = dataset
        self.oversample_factor = oversample_factor
        self.truth_indices = dataset.truth_indices * oversample_factor  # Oversample truths
        self.lie_indices = dataset.lie_indices
        
        min_samples = max(1000, len(self.truth_indices))
        target_size = min(len(self.truth_indices), len(self.lie_indices))
        target_size = max(target_size, min_samples)
        
        if len(self.truth_indices) < target_size:
            self.truth_indices = self.truth_indices * (target_size // len(self.truth_indices) + 1)
        if len(self.lie_indices) < target_size:
            self.lie_indices = self.lie_indices * (target_size // len(self.lie_indices) + 1)
        
        self.truth_indices = random.sample(self.truth_indices, target_size)
        self.lie_indices = random.sample(self.lie_indices, target_size)
        
        self.indices = self.truth_indices + self.lie_indices
        random.shuffle(self.indices)
    
    def __iter__(self):
        return iter(self.indices)
    
    def __len__(self):
        return len(self.indices)

In [10]:
class SimpleAttention(nn.Module):
    def __init__(self, hidden_size, dropout=0.1):
        super(SimpleAttention, self).__init__()
        self.hidden_size = hidden_size
        self.query_proj = nn.Linear(hidden_size, hidden_size)
        self.key_proj = nn.Linear(hidden_size, hidden_size)
        self.value_proj = nn.Linear(hidden_size, hidden_size)
        self.out_proj = nn.Linear(hidden_size, hidden_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, query, key, value, mask=None):
        query = self.query_proj(query)
        key = self.key_proj(key)
        value = self.value_proj(value)
        scores = torch.matmul(query, key.transpose(-2, -1)) / (self.hidden_size ** 0.5)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context = torch.matmul(attn_weights, value)
        output = self.out_proj(context)
        return output

class GraphAttentionLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout=0.1, alpha=0.2):
        super(GraphAttentionLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.alpha = alpha
        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)
        self.leakyrelu = nn.LeakyReLU(self.alpha)
        
    def forward(self, features, adj_matrix):
        h = torch.mm(features, self.W)  # (N, out_features)
        N = h.size()[0]
        a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1)
        a_input = a_input.view(N, N, 2 * self.out_features)
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))
        zero_vec = -9e15 * torch.ones_like(e)
        attention = torch.where(adj_matrix > 0, e, zero_vec)
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime = torch.matmul(attention, h)
        return h_prime

class ContextEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=1, bidirectional=True, dropout=0.1):
        super(ContextEncoder, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, 
                            bidirectional=bidirectional, dropout=dropout if num_layers > 1 else 0)
        self.output_dim = hidden_dim * (2 if bidirectional else 1)

    def forward(self, token_embeddings, attention_mask):
        lengths = torch.clamp(attention_mask.sum(dim=1), min=1).cpu()
        packed = nn.utils.rnn.pack_padded_sequence(token_embeddings, lengths, batch_first=True, enforce_sorted=False)
        packed_outputs, (h_n, _) = self.lstm(packed)
        num_directions = 2 if self.lstm.bidirectional else 1
        h_n = h_n.view(self.lstm.num_layers, num_directions, token_embeddings.size(0), self.lstm.hidden_size)
        h_final = torch.cat((h_n[-1, 0, :, :], h_n[-1, 1, :, :]), dim=1)
        return h_final

class ImprovedDeceptionModel(nn.Module):
    def __init__(self, model_name, use_game_scores=True):
        super(ImprovedDeceptionModel, self).__init__()
        self.transformer = RobertaModel.from_pretrained(model_name)
        self.context_transformer = RobertaModel.from_pretrained(model_name)
        
        for layer in self.transformer.encoder.layer[:2]:
            for param in layer.parameters():
                param.requires_grad = False
        for layer in self.context_transformer.encoder.layer[:2]:
            for param in layer.parameters():
                param.requires_grad = False
        
        self.hidden_size = self.transformer.config.hidden_size
        self.use_game_scores = use_game_scores
        self.conceptnet_dim = 300  # Size of Numberbatch embeddings
        
        self.dropout = nn.Dropout(0.3)
        self.context_encoder = ContextEncoder(input_dim=self.hidden_size, hidden_dim=self.hidden_size//2, 
                                              num_layers=1, bidirectional=True, dropout=0.1)
        
        # Adjust input dimension for graph attention
        self.combined_dim = self.hidden_size + self.context_encoder.output_dim + self.conceptnet_dim
        self.gat1 = GraphAttentionLayer(self.combined_dim, 256)
        self.gat2 = GraphAttentionLayer(256, 256)
        
        if use_game_scores:
            self.score_proj = nn.Sequential(
                nn.Linear(1, 32),
                nn.LayerNorm(32),
                nn.ReLU(),
                nn.Dropout(0.2)
            )
            fusion_dim = self.combined_dim + 256 + 32
        else:
            fusion_dim = self.combined_dim + 256
        
        self.feature_norm = nn.LayerNorm(fusion_dim)
        self.classifier1 = nn.Linear(fusion_dim, 2)
        self.classifier2 = nn.Linear(self.combined_dim, 2)
        self.classifier3 = nn.Linear(256, 2)
        self.ensemble_weights = nn.Parameter(torch.ones(3))
    
    def forward(self, input_ids, attention_mask, context_input_ids=None, 
                context_attention_mask=None, game_scores=None, batch_adj_matrix=None, conceptnet_features=None):
        outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        text_features = self.dropout(outputs.last_hidden_state[:, 0, :])
        
        if context_input_ids is not None and torch.sum(context_input_ids) > 0:
            ctx_outputs = self.context_transformer(input_ids=context_input_ids, attention_mask=context_attention_mask)
            context_summary = self.context_encoder(ctx_outputs.last_hidden_state, context_attention_mask)
        else:
            context_summary = torch.zeros(text_features.size(0), self.context_encoder.output_dim, device=text_features.device)
        
        # Include ConceptNet features
        combined_features = torch.cat([text_features, context_summary, conceptnet_features], dim=1)
        
        if batch_adj_matrix is not None:
            graph_features = self.gat1(combined_features, batch_adj_matrix)
            graph_features = F.elu(graph_features)
            graph_features = self.gat2(graph_features, batch_adj_matrix)
        else:
            graph_features = torch.zeros(combined_features.size(0), 256, device=combined_features.device)
        
        if self.use_game_scores and game_scores is not None:
            score_features = self.score_proj(game_scores.unsqueeze(1))
            all_features = torch.cat([combined_features, graph_features, score_features], dim=1)
        else:
            all_features = torch.cat([combined_features, graph_features], dim=1)
        
        all_features = self.feature_norm(all_features)
        logits1 = self.classifier1(all_features)
        logits2 = self.classifier2(combined_features)
        logits3 = self.classifier3(graph_features)
        ensemble_weights = F.softmax(self.ensemble_weights, dim=0)
        final_logits = (ensemble_weights[0] * logits1 +
                        ensemble_weights[1] * logits2 +
                        ensemble_weights[2] * logits3)
        return final_logits

In [11]:
class FocalWeightedLoss(nn.Module):
    def __init__(self, class_weights, truth_focal_weight=4.0):
        super(FocalWeightedLoss, self).__init__()
        self.class_weights = class_weights
        self.truth_focal_weight = truth_focal_weight
    
    def forward(self, logits, targets):
        ce_loss = F.cross_entropy(logits, targets, weight=self.class_weights, reduction='none')
        probs = F.softmax(logits, dim=1)
        truth_probs = probs[:, 0]  # Probability for truth class
        truth_mask = (targets == 0).float()
        focal_weight = (1 - truth_probs) ** self.truth_focal_weight
        focal_loss = truth_mask * focal_weight * ce_loss + (1 - truth_mask) * ce_loss
        return focal_loss.mean()

def custom_collate_fn(batch):
    # Collate all keys except 'relative_positions'
    batch_without_relative = [{k: v for k, v in item.items() if k != 'relative_positions'} for item in batch]
    collated = torch.utils.data.dataloader.default_collate(batch_without_relative)
    collated['relative_positions'] = [item['relative_positions'] for item in batch]
    return collated

def prepare_batch_for_model(batch, device):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    context_input_ids = batch['context_input_ids'].to(device) if 'context_input_ids' in batch else None
    context_attention_mask = batch['context_attention_mask'].to(device) if 'context_attention_mask' in batch else None
    labels = batch['label'].to(device)
    scores = batch['score'].to(device) if 'score' in batch else None
    conceptnet_features = batch['conceptnet_features'].to(device)
    batch_size = input_ids.size(0)
    
    batch_adj_matrix = torch.zeros((batch_size, batch_size), device=device)
    conv_ids = batch['conv_id'] if isinstance(batch['conv_id'], list) else batch['conv_id'].tolist()
    positions = batch['position'] if isinstance(batch['position'], list) else batch['position'].tolist()
    for i in range(batch_size):
        for j in range(batch_size):
            if conv_ids[i] == conv_ids[j]:
                if i == j:
                    batch_adj_matrix[i, j] = 1.0
                else:
                    distance = abs(positions[i] - positions[j])
                    batch_adj_matrix[i, j] = 1.0 / (distance + 1)
    
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'context_input_ids': context_input_ids,
        'context_attention_mask': context_attention_mask,
        'labels': labels,
        'scores': scores,
        'batch_adj_matrix': batch_adj_matrix,
        'conceptnet_features': conceptnet_features
    }


In [ ]:

def train(model, dataloader, optimizer, scheduler, device, class_weights, truth_focal_weight=4.0, gradient_accumulation_steps=1):
    model.train()
    total_loss = 0
    all_labels = []
    all_preds = []
    loss_fn = FocalWeightedLoss(class_weights, truth_focal_weight)
    optimizer.zero_grad()
    accumulated_steps = 0
    
    for batch in tqdm(dataloader, desc="Training"):
        batch_data = prepare_batch_for_model(batch, device)
        outputs = model(
            input_ids=batch_data['input_ids'], 
            attention_mask=batch_data['attention_mask'],
            context_input_ids=batch_data['context_input_ids'],
            context_attention_mask=batch_data['context_attention_mask'],
            game_scores=batch_data['scores'],
            batch_adj_matrix=batch_data['batch_adj_matrix'],
            conceptnet_features=batch_data['conceptnet_features']
        )
        loss = loss_fn(outputs, batch_data['labels'])
        loss = loss / gradient_accumulation_steps
        loss.backward()
        total_loss += loss.item() * gradient_accumulation_steps
        _, preds = torch.max(outputs, dim=1)
        all_labels.extend(batch_data['labels'].cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
        accumulated_steps += 1
        if accumulated_steps % gradient_accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            if scheduler:
                scheduler.step()
            optimizer.zero_grad()
    
    if accumulated_steps % gradient_accumulation_steps != 0:
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        if scheduler:
            scheduler.step()
    
    avg_loss = total_loss / len(dataloader)
    truth_f1 = f1_score(all_labels, all_preds, pos_label=0, average='binary', zero_division=0)
    lie_f1 = f1_score(all_labels, all_preds, pos_label=1, average='binary', zero_division=0)
    macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    cm = confusion_matrix(all_labels, all_preds)
    return avg_loss, truth_f1, lie_f1, macro_f1, cm

def evaluate(model, dataloader, device, class_weights, truth_focal_weight=4.0):
    model.eval()
    total_loss = 0
    all_labels = []
    all_preds = []
    loss_fn = FocalWeightedLoss(class_weights, truth_focal_weight)
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            batch_data = prepare_batch_for_model(batch, device)
            outputs = model(
                input_ids=batch_data['input_ids'], 
                attention_mask=batch_data['attention_mask'],
                context_input_ids=batch_data['context_input_ids'],
                context_attention_mask=batch_data['context_attention_mask'],
                game_scores=batch_data['scores'],
                batch_adj_matrix=batch_data['batch_adj_matrix'],
                conceptnet_features=batch_data['conceptnet_features']
            )
            loss = loss_fn(outputs, batch_data['labels'])
            total_loss += loss.item()
            _, preds = torch.max(outputs, dim=1)
            all_labels.extend(batch_data['labels'].cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    
    avg_loss = total_loss / len(dataloader)
    truth_f1 = f1_score(all_labels, all_preds, pos_label=0, average='binary', zero_division=0)
    lie_f1 = f1_score(all_labels, all_preds, pos_label=1, average='binary', zero_division=0)
    macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    print(classification_report(all_labels, all_preds, target_names=['Truth', 'Lie'], digits=4, zero_division=0))
    cm = confusion_matrix(all_labels, all_preds)
    return avg_loss, truth_f1, lie_f1, macro_f1, cm

def plot_confusion_matrix(cm, epoch, split='val'):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Truth', 'Lie'], yticklabels=['Truth', 'Lie'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix - {split.capitalize()} (Epoch {epoch+1})')
    plt.savefig(f'confusion_matrix_{split}_epoch_{epoch+1}.png')
    plt.close()

def plot_metrics(train_metric, val_metric, metric_name):
    plt.figure(figsize=(10, 6))
    epochs = range(1, len(train_metric) + 1)
    plt.plot(epochs, train_metric, 'b-', label=f'Train {metric_name}')
    plt.plot(epochs, val_metric, 'r-', label=f'Val {metric_name}')
    plt.title(f'{metric_name} over Training')
    plt.xlabel('Epoch')
    plt.ylabel(metric_name)
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{metric_name.lower()}_plot.png')
    plt.close()

def main():
    try:
        print("Loading tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL)
        
        print("Loading datasets...")
        train_dataset = EnhancedDeceptionDataset(TRAIN_PATH, tokenizer, use_game_scores=USE_GAME_SCORES)
        val_dataset = EnhancedDeceptionDataset(VAL_PATH, tokenizer, use_game_scores=USE_GAME_SCORES)
        test_dataset = EnhancedDeceptionDataset(TEST_PATH, tokenizer, use_game_scores=USE_GAME_SCORES)
        
        train_sampler = EnhancedBalancedSampler(train_dataset, oversample_factor=OVERSAMPLING_FACTOR)
        train_loader = DataLoader(
            train_dataset,
            batch_size=BATCH_SIZE,
            sampler=train_sampler,
            num_workers=2,
            collate_fn=custom_collate_fn
        )
        val_loader = DataLoader(
            val_dataset,
            batch_size=BATCH_SIZE,
            shuffle=False,
            num_workers=2,
            collate_fn=custom_collate_fn
        )
        test_loader = DataLoader(
            test_dataset,
            batch_size=BATCH_SIZE,
            shuffle=False,
            num_workers=2,
            collate_fn=custom_collate_fn
        )
        
        train_class_counts = train_dataset.class_counts
        total_samples = sum(train_class_counts.values())
        weight_0 = total_samples / (train_class_counts.get(0, 1) * 2)
        weight_1 = total_samples / (train_class_counts.get(1, 1) * 2)
        class_weights = torch.tensor([weight_0, weight_1], dtype=torch.float).to(DEVICE)
        print(f"Class weights: Truth = {weight_0:.4f}, Lie = {weight_1:.4f}")
        
        print("Initializing improved model...")
        model = ImprovedDeceptionModel(TRANSFORMER_MODEL, use_game_scores=USE_GAME_SCORES).to(DEVICE)
        
        optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=0.01)
        total_steps = len(train_loader) * EPOCHS // GRADIENT_ACCUMULATION_STEPS
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=int(total_steps * 0.1),
            num_training_steps=total_steps
        )
        
        print(f"Starting training for {EPOCHS} epochs...")
        best_truth_f1 = 0
        best_macro_f1 = 0
        no_improvement_count = 0
        
        train_losses = []
        train_truth_f1s = []
        train_lie_f1s = []
        train_macro_f1s = []
        val_losses = []
        val_truth_f1s = []
        val_lie_f1s = []
        val_macro_f1s = []
        
        for epoch in range(EPOCHS):
            train_loss, train_truth_f1, train_lie_f1, train_macro_f1, train_cm = train(
                model, train_loader, optimizer, scheduler, DEVICE, class_weights,
                truth_focal_weight=TRUTH_FOCAL_WEIGHT,
                gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS
            )
            val_loss, val_truth_f1, val_lie_f1, val_macro_f1, val_cm = evaluate(
                model, val_loader, DEVICE, class_weights,
                truth_focal_weight=TRUTH_FOCAL_WEIGHT
            )
            
            train_losses.append(train_loss)
            train_truth_f1s.append(train_truth_f1)
            train_lie_f1s.append(train_lie_f1)
            train_macro_f1s.append(train_macro_f1)
            
            val_losses.append(val_loss)
            val_truth_f1s.append(val_truth_f1)
            val_lie_f1s.append(val_lie_f1)
            val_macro_f1s.append(val_macro_f1)
            
            print(f"Epoch {epoch+1}/{EPOCHS}")
            print(f"Train - Loss: {train_loss:.4f}, Truth F1: {train_truth_f1:.4f}, Lie F1: {train_lie_f1:.4f}, Macro F1: {train_macro_f1:.4f}")
            print(f"Val   - Loss: {val_loss:.4f}, Truth F1: {val_truth_f1:.4f}, Lie F1: {val_lie_f1:.4f}, Macro F1: {val_macro_f1:.4f}")
            print("Confusion Matrix (Val):")
            print(val_cm)
            print("-" * 50)
            plot_confusion_matrix(val_cm, epoch, 'val')
            
            improved = False
            if val_truth_f1 > best_truth_f1:
                best_truth_f1 = val_truth_f1
                torch.save(model.state_dict(), 'best_truth_f1_model.pt')
                print(f"Saved new best model with Truth F1: {val_truth_f1:.4f}")
                improved = True
            if val_macro_f1 > best_macro_f1:
                best_macro_f1 = val_macro_f1
                torch.save(model.state_dict(), 'best_macro_f1_model.pt')
                print(f"Saved new best model with Macro F1: {val_macro_f1:.4f}")
                improved = True
            
            if not improved:
                no_improvement_count += 1
                if no_improvement_count >= EARLY_STOPPING_PATIENCE:
                    print(f"Early stopping triggered after {epoch+1} epochs")
                    break
            else:
                no_improvement_count = 0
        
        plot_metrics(train_losses, val_losses, 'Loss')
        plot_metrics(train_truth_f1s, val_truth_f1s, 'Truth F1')
        plot_metrics(train_lie_f1s, val_lie_f1s, 'Lie F1')
        plot_metrics(train_macro_f1s, val_macro_f1s, 'Macro F1')
        
        print("\nEvaluating best model (by Truth F1) on test set:")
        model.load_state_dict(torch.load('best_truth_f1_model.pt'))
        test_loss, test_truth_f1, test_lie_f1, test_macro_f1, test_cm = evaluate(
            model, test_loader, DEVICE, class_weights,
            truth_focal_weight=TRUTH_FOCAL_WEIGHT
        )
        print(f"\nTest Results - Truth F1 Model:")
        print(f"Loss: {test_loss:.4f}, Truth F1: {test_truth_f1:.4f}, Lie F1: {test_lie_f1:.4f}, Macro F1: {test_macro_f1:.4f}")
        print("Confusion Matrix:")
        print(test_cm)
        
        print("\nEvaluating best model (by Macro F1) on test set:")
        model.load_state_dict(torch.load('best_macro_f1_model.pt'))
        test_loss, test_truth_f1, test_lie_f1, test_macro_f1, test_cm = evaluate(
            model, test_loader, DEVICE, class_weights,
            truth_focal_weight=TRUTH_FOCAL_WEIGHT
        )
        print(f"\nTest Results - Macro F1 Model:")
        print(f"Loss: {test_loss:.4f}, Truth F1: {test_truth_f1:.4f}, Lie F1: {test_lie_f1:.4f}, Macro F1: {test_macro_f1:.4f}")
        print("Confusion Matrix:")
        print(test_cm)
        
    except Exception as e:
        print(f"An error occurred during execution: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

2025-04-12 18:01:26.408568: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744480886.615921      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744480886.673489      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda
Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

              precision    recall  f1-score   support

       Truth     0.1042    0.0893    0.0962        56
         Lie     0.9627    0.9684    0.9655      1360

    accuracy                         0.9336      1416
   macro avg     0.5334    0.5288    0.5308      1416
weighted avg     0.9288    0.9336    0.9312      1416

Epoch 5/5
Train - Loss: 0.0221, Truth F1: 0.9923, Lie F1: 0.9923, Macro F1: 0.9923
Val   - Loss: 2.3023, Truth F1: 0.0962, Lie F1: 0.9655, Macro F1: 0.5308
Confusion Matrix (Val):
[[   5   51]
 [  43 1317]]
--------------------------------------------------

Evaluating best model (by Truth F1) on test set:


/tmp/ipykernel_31/3032531545.py:633: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_truth_f1_model.pt'))
Evaluating: 100%|██████████| 8

              precision    recall  f1-score   support

       Truth     0.1421    0.5917    0.2292       240
         Lie     0.9437    0.6573    0.7749      2501

    accuracy                         0.6516      2741
   macro avg     0.5429    0.6245    0.5021      2741
weighted avg     0.8736    0.6516    0.7271      2741


Test Results - Truth F1 Model:
Loss: 0.4124, Truth F1: 0.2292, Lie F1: 0.7749, Macro F1: 0.5021
Confusion Matrix:
[[ 142   98]
 [ 857 1644]]

Evaluating best model (by Macro F1) on test set:


Evaluating: 100%|██████████| 86/86 [00:20<00:00,  4.22it/s]


              precision    recall  f1-score   support

       Truth     0.2235    0.1667    0.1909       240
         Lie     0.9219    0.9444    0.9330      2501

    accuracy                         0.8763      2741
   macro avg     0.5727    0.5555    0.5620      2741
weighted avg     0.8608    0.8763    0.8681      2741


Test Results - Macro F1 Model:
Loss: 1.8916, Truth F1: 0.1909, Lie F1: 0.9330, Macro F1: 0.5620
Confusion Matrix:
[[  40  200]
 [ 139 2362]]


# INFERENCE CODE

In [14]:
def evaluate(model, dataloader, device, class_weights, truth_focal_weight=4.0):
    model.eval()
    total_loss = 0
    all_labels = []
    all_preds = []
    loss_fn = FocalWeightedLoss(class_weights, truth_focal_weight)
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            batch_data = prepare_batch_for_model(batch, device)
            outputs = model(
                input_ids=batch_data['input_ids'], 
                attention_mask=batch_data['attention_mask'],
                context_input_ids=batch_data['context_input_ids'],
                context_attention_mask=batch_data['context_attention_mask'],
                game_scores=batch_data['scores'],
                batch_adj_matrix=batch_data['batch_adj_matrix'],
                conceptnet_features=batch_data['conceptnet_features']
            )
            loss = loss_fn(outputs, batch_data['labels'])
            total_loss += loss.item()
            _, preds = torch.max(outputs, dim=1)
            all_labels.extend(batch_data['labels'].cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    
    avg_loss = total_loss / len(dataloader)
    truth_f1 = f1_score(all_labels, all_preds, pos_label=0, average='binary', zero_division=0)
    lie_f1 = f1_score(all_labels, all_preds, pos_label=1, average='binary', zero_division=0)
    macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    print(classification_report(all_labels, all_preds, target_names=['Truth', 'Lie'], digits=4, zero_division=0))
    cm = confusion_matrix(all_labels, all_preds)
    return avg_loss, truth_f1, lie_f1, macro_f1, cm

def plot_confusion_matrix(cm, epoch, split='val'):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Truth', 'Lie'], yticklabels=['Truth', 'Lie'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix - {split.capitalize()} (Epoch {epoch+1})')
    plt.savefig(f'confusion_matrix_{split}_epoch_{epoch+1}.png')
    plt.close()

def plot_metrics(train_metric, val_metric, metric_name):
    plt.figure(figsize=(10, 6))
    epochs = range(1, len(train_metric) + 1)
    plt.plot(epochs, train_metric, 'b-', label=f'Train {metric_name}')
    plt.plot(epochs, val_metric, 'r-', label=f'Val {metric_name}')
    plt.title(f'{metric_name} over Training')
    plt.xlabel('Epoch')
    plt.ylabel(metric_name)
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{metric_name.lower()}_plot.png')
    plt.close()
    
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL)

print("Loading datasets...")
train_dataset = EnhancedDeceptionDataset(TRAIN_PATH, tokenizer, use_game_scores=USE_GAME_SCORES)
val_dataset = EnhancedDeceptionDataset(VAL_PATH, tokenizer, use_game_scores=USE_GAME_SCORES)
test_dataset = EnhancedDeceptionDataset(TEST_PATH, tokenizer, use_game_scores=USE_GAME_SCORES)

train_sampler = EnhancedBalancedSampler(train_dataset, oversample_factor=OVERSAMPLING_FACTOR)
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    sampler=train_sampler,
    num_workers=2,
    collate_fn=custom_collate_fn
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    collate_fn=custom_collate_fn
)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    collate_fn=custom_collate_fn
)

train_class_counts = train_dataset.class_counts
total_samples = sum(train_class_counts.values())
weight_0 = total_samples / (train_class_counts.get(0, 1) * 2)
weight_1 = total_samples / (train_class_counts.get(1, 1) * 2)
class_weights = torch.tensor([weight_0, weight_1], dtype=torch.float).to(DEVICE)
print(f"Class weights: Truth = {weight_0:.4f}, Lie = {weight_1:.4f}")

print("Initializing improved model...")
model = ImprovedDeceptionModel(TRANSFORMER_MODEL, use_game_scores=USE_GAME_SCORES).to(DEVICE)

print("\nEvaluating best model (by Truth F1) on test set:")
model.load_state_dict(torch.load('/kaggle/input/novel_models_conceptnet/pytorch/default/1/best_truth_f1_model.pt'))
test_loss, test_truth_f1, test_lie_f1, test_macro_f1, test_cm = evaluate(
    model, test_loader, DEVICE, class_weights,
    truth_focal_weight=TRUTH_FOCAL_WEIGHT
)
print(f"\nTest Results - Truth F1 Model:")
print(f"Loss: {test_loss:.4f}, Truth F1: {test_truth_f1:.4f}, Lie F1: {test_lie_f1:.4f}, Macro F1: {test_macro_f1:.4f}")
print("Confusion Matrix:")
print(test_cm)

print("\nEvaluating best model (by Macro F1) on test set:")
model.load_state_dict(torch.load('/kaggle/input/novel_models_conceptnet/pytorch/default/1/best_macro_f1_model.pt'))
test_loss, test_truth_f1, test_lie_f1, test_macro_f1, test_cm = evaluate(
    model, test_loader, DEVICE, class_weights,
    truth_focal_weight=TRUTH_FOCAL_WEIGHT
)
print(f"\nTest Results - Macro F1 Model:")
print(f"Loss: {test_loss:.4f}, Truth F1: {test_truth_f1:.4f}, Lie F1: {test_lie_f1:.4f}, Macro F1: {test_macro_f1:.4f}")
print("Confusion Matrix:")
print(test_cm)

Loading tokenizer...
Loading datasets...
Dataset loaded from /kaggle/input/nlp-proj-data/Data/train.jsonl
Total messages: 13132
Truth: 591 (4.50%)
Lie: 12541 (95.50%)
Dataset loaded from /kaggle/input/nlp-proj-data/Data/validation.jsonl
Total messages: 1416
Truth: 56 (3.95%)
Lie: 1360 (96.05%)
Dataset loaded from /kaggle/input/nlp-proj-data/Data/test.jsonl
Total messages: 2741
Truth: 240 (8.76%)
Lie: 2501 (91.24%)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Class weights: Truth = 11.1100, Lie = 0.5236
Initializing improved model...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Evaluating best model (by Truth F1) on test set:


/tmp/ipykernel_31/626108610.py:99: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/input/novel_models_conceptnet/pytorch/default/1/be

              precision    recall  f1-score   support

       Truth     0.1421    0.5917    0.2292       240
         Lie     0.9437    0.6573    0.7749      2501

    accuracy                         0.6516      2741
   macro avg     0.5429    0.6245    0.5021      2741
weighted avg     0.8736    0.6516    0.7271      2741


Test Results - Truth F1 Model:
Loss: 0.4124, Truth F1: 0.2292, Lie F1: 0.7749, Macro F1: 0.5021
Confusion Matrix:
[[ 142   98]
 [ 857 1644]]

Evaluating best model (by Macro F1) on test set:


Evaluating: 100%|██████████| 86/86 [00:20<00:00,  4.16it/s]

              precision    recall  f1-score   support

       Truth     0.2235    0.1667    0.1909       240
         Lie     0.9219    0.9444    0.9330      2501

    accuracy                         0.8763      2741
   macro avg     0.5727    0.5555    0.5620      2741
weighted avg     0.8608    0.8763    0.8681      2741


Test Results - Macro F1 Model:
Loss: 1.8916, Truth F1: 0.1909, Lie F1: 0.9330, Macro F1: 0.5620
Confusion Matrix:
[[  40  200]
 [ 139 2362]]


In [ ]:
! zip -r /kaggle/working/archive.zip /kaggle/working/

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/best_macro_f1_model.pt